In [21]:
import requests
import os
from PIL import Image
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
from io import BytesIO

import pandas as pd

In [22]:
from DB_and_Azure import sql_db_functions as SQLf, azure_functions as Af

## Creating functions

In [23]:
def create_soup(url):

    ua = UserAgent()
    header = {'User-Agent':str(ua.chrome)}

    # Send an HTTP request to the URL
    response = requests.get(url, headers = header)
    soup = BeautifulSoup(response.content, 'html.parser')

    return soup


In [24]:
def contains_required_number(line):
    
    keeping = ['http','http']
    # 001: Prod
    # 004: back
    # 005: 3/4 image
    return any(num in line for num in keeping)

In [25]:
def get_image_links(prod_soup):
    
    item_list_1 = prod_soup.select('div div div div div ul li button div img')

    ##### get image
    images = []
    i = 0
    for item in item_list_1: 
            
        if (i == len(item_list_1)-3) | (i == len(item_list_1)-1):
            images = images + [item['src']]

        i += 1
    
    return images

In [26]:
def get_description(prod_soup):
    
    description_list = prod_soup.select('dd')

    #Get description
    description = ''
    for d in description_list:
        description = description + ' ' + d.text

    #Get Materials
    description_list = prod_soup.find_all(class_ = 'f8f91e f02838' )

    description = description + ' Materials: '

    for d in description_list[0:2]:
        description = description + ' ' + d.text

    return description
    

In [27]:
def links_to_images(links,prod_name,container_client):
    ua = UserAgent()
    header = {'User-Agent':str(ua.chrome)}

    i=0
    for img_link in links:
        response = requests.get(img_link, headers = header)
        img = Image.open(BytesIO(response.content))

        # Resize the image to 600
        img_resized = img.resize((500, 500))

        img_name = f"HM{prod_name}_{i}.jpg"

        # Load to Blob
        Af.images_to_blob(image = img_resized,image_name = img_name,container_client = container_client)
        #print(img_link)
        #print(img_name)
        i+=1



In [28]:
import re

def get_price(prod_soup):

    text = prod_soup.select('span.edbe20')[0].text


    # Remove any non-numeric characters except for ',' and '.'
    cleaned_text = re.sub(r'[^\d,\.]', '', text)
    
    # Replace comma with a period if there's no period already (to handle decimal part)
    if ',' in cleaned_text and '.' not in cleaned_text:
        cleaned_text = cleaned_text.replace(',', '.')
    elif ',' in cleaned_text and '.' in cleaned_text:
        # If both ',' and '.' are present, keep only the period as the decimal separator
        cleaned_text = cleaned_text.replace(',', '')
    
    # Convert the string to a float
    number = float(cleaned_text)
    
    return number

In [29]:
def HM(category_url,n_products):

    soup = create_soup(category_url)
    item_list = soup.select('li section article div div div div ul li a')

    container_client = Af.connect_blob()
    
    conn, cursor = SQLf.connect_sql()

    br = 0
    pass_i = 0


    for item in item_list:     

        pass_i +=1

        if br == n_products:
            break
        elif pass_i == 5:

            #Scrapp
            prod_soup = create_soup(item['href'])

            prod_images_links = get_image_links(prod_soup=prod_soup)
            prod_description = get_description(prod_soup=prod_soup)
            prod_price = get_price(prod_soup=prod_soup)

            prod_name = 'HM_'+str(br)  
            
            # Links to image, load to blob and return prod_images_names
            prod_images_names = links_to_images(links = prod_images_links,prod_name= br, container_client = container_client)
            
            SQLf.insert_description_image_to_db(conn = conn, brand='HM', descript=prod_description, price= prod_price,image_names=prod_images_names)

            pass_i = 0


        br += 1


In [30]:
# Define the URL you want to scrape
url = 'https://www2.hm.com/it_it/donna/acquista-per-prodotto/top.html'


# Need to have # of assets times 6
HM(url,192)


Error: unknown file extension: 
Error: unknown file extension: 
Error occurred: 'NoneType' object is not iterable
Error: unknown file extension: 
Error: unknown file extension: 
Error occurred: 'NoneType' object is not iterable
Error: unknown file extension: 
Error: unknown file extension: 
Error occurred: 'NoneType' object is not iterable
Error: unknown file extension: 
Error: unknown file extension: 
Error occurred: 'NoneType' object is not iterable
Error: unknown file extension: 
Error: unknown file extension: 
Error occurred: 'NoneType' object is not iterable
Error: unknown file extension: 
Error: unknown file extension: 
Error occurred: 'NoneType' object is not iterable
Error: unknown file extension: 
Error: unknown file extension: 
Error occurred: 'NoneType' object is not iterable
Error: unknown file extension: 
Error: unknown file extension: 
Error occurred: 'NoneType' object is not iterable


KeyboardInterrupt: 